In [1]:
# Necessary libraries for this project.  We will be using PuLP for our linear programming:
from pulp import *
import pandas as pd

In [2]:
# I couldn't think of a smooth way to convert a DOCX into meaningful data, so I am going to build these manually

hubs = ['CVG', 'AFW']
focus_cities = ['Leipzig', 'San Bernardino']  # Per professor, we are not treating 'Hyderabad' as a focus city, but rather as a center
centers = ['Hyderabad','Paris','Cologne','Hanover','Bengaluru','Coimbatore','Delhi','Mumbai','Cagliari','Catania','Milan','Rome','Katowice','Barcelona','Madrid','Castle Donington','London',
           'Mobile','Anchorage','Fairbanks','Phoenix','Los Angeles','Ontario','Riverside','Sacramento','San Francisco','Stockton','Denver','Hartford','Miami','Lakeland','Tampa','Atlanta',
           'Honolulu','Kahului/Maui','Kona','Chicago','Rockford','Fort Wayne','South Bend','Des Moines','Wichita','New Orleans','Baltimore','Minneapolis','Kansas City','St. Louis',
           'Omaha','Manchester','Albuquerque','New York','Charlotte','Toledo','Wilmington','Portland','Allentown','Pittsburgh','San Juan','Nashville','Austin','Dallas/Fort Worth','Houston',
           'San Antonio','Richmond','Seattle/Tacoma','Spokane']
print(len(centers)) # Got all 66

hub_capacity = {'CVG': 95650, 'AFW': 44350}
focus_capacity = {'Leipzig': 85000, 'San Bernardino': 36000}

demand = {
    'Hyderabad': 0,'Paris': 6500 ,'Cologne': 640 ,'Hanover': 180 ,'Bengaluru': 9100 ,'Coimbatore': 570 ,'Delhi': 19000 ,'Mumbai': 14800 ,'Cagliari': 90 ,'Catania': 185 ,'Milan': 800 ,'Rome': 1700 ,'Katowice': 170 ,
    'Barcelona': 2800 ,'Madrid': 3700 ,'Castle Donington': 30 ,'London': 6700 ,'Mobile': 190 ,'Anchorage': 175 ,'Fairbanks': 38 ,'Phoenix': 2400 ,'Los Angeles': 7200 ,'Ontario': 100 ,'Riverside': 1200 ,
    'Sacramento': 1100 ,'San Francisco': 1900 ,'Stockton': 240 ,'Denver': 1500 ,'Hartford': 540 ,'Miami': 3400 ,'Lakeland': 185 ,'Tampa': 1600 ,'Atlanta': 3000 ,'Honolulu': 500 ,'Kahului/Maui': 16 ,
    'Kona': 63 ,'Chicago': 5100 ,'Rockford': 172 ,'Fort Wayne': 200 ,'South Bend': 173 ,'Des Moines': 300 ,'Wichita': 290 ,'New Orleans': 550 ,'Baltimore': 1300 ,'Minneapolis': 1700 ,'Kansas City': 975 ,
    'St. Louis': 1200 ,'Omaha': 480 ,'Manchester': 100 ,'Albuquerque': 450 ,'New York': 11200 ,'Charlotte': 900 ,'Toledo': 290 ,'Wilmington': 150 ,'Portland': 1200 ,'Allentown': 420 ,'Pittsburgh': 1000 ,
    'San Juan': 1100 ,'Nashville': 650 ,'Austin': 975 ,'Dallas/Fort Worth': 3300 ,'Houston': 3300 ,'San Antonio': 1100 ,'Richmond': 600 ,'Seattle/Tacoma': 2000 ,'Spokane': 260 
}

# Hub to focus city costs (Our X variables)
costs_x = {
    ('CVG', 'Leipzig'): 1.5, ('CVG', 'San Bernardino'): 0.5, ('AFW', 'San Bernardino'): 0.5
}

# Hub to center directly costs (Our y variables)
costs_y = {
    ('CVG', 'Paris'): 1.6, ('CVG', 'Cologne'): 1.5, ('CVG', 'Hanover'): 1.5,('CVG', 'Cagliari'): 1.5,('CVG', 'Catania'): 1.5,('CVG', 'Milan'): 1.5,('CVG', 'Rome'): 1.5,('CVG', 'Katowice'): 1.4,('CVG', 'Barcelona'): 1.5,		
    ('CVG', 'Madrid'): 1.6,('CVG', 'Castle Donington'): 1.4,('CVG', 'London'): 1.6,('CVG', 'Mobile'): 0.5,	('AFW', 'Mobile'): 0.5,('CVG', 'Anchorage'): 1.3,('AFW', 'Anchorage'): 1,('CVG', 'Fairbanks'): 1.4,	
    ('AFW', 'Fairbanks'): 1,('CVG', 'Phoenix'): 0.5,	('AFW', 'Phoenix'): 0.5,('CVG', 'Los Angeles'): 0.5,('AFW', 'Los Angeles'): 0.5,('CVG', 'Ontario'): 0.5,('AFW', 'Ontario'): 0.5,('CVG', 'Riverside'): 0.5,	
    ('AFW', 'Riverside'): 0.5,('CVG', 'Sacramento'): 0.5,('AFW', 'Sacramento'): 0.5,('CVG', 'San Francisco'): 0.5,('AFW', 'San Francisco'): 0.5,('CVG', 'Stockton'): 0.5,('AFW', 'Stockton'): 0.5,('CVG', 'Denver'): 0.5,	
    ('AFW', 'Denver'): 0.5,	('CVG', 'Hartford'): 0.5,('AFW', 'Hartford'): 0.5,('CVG', 'Miami'): 0.5,('AFW', 'Miami'): 0.5,('CVG', 'Lakeland'): 0.5,('AFW', 'Lakeland'): 0.5,('CVG', 'Tampa'): 0.5,('AFW', 'Tampa'): 0.5,	
    ('CVG', 'Atlanta'): 0.5,('AFW', 'Atlanta'): 0.5,('AFW', 'Honolulu'): 0.5,('AFW', 'Kahului/Maui'): 0.5,('AFW', 'Kona'): 0.5,('CVG', 'Chicago'): 0.5,	('AFW', 'Chicago'): 0.5,('CVG', 'Rockford'): 0.5,('AFW', 'Rockford'): 0.5,	
    ('CVG', 'Fort Wayne'): 0.5,('AFW', 'Fort Wayne'): 0.5,('CVG', 'South Bend'): 0.5,('AFW', 'South Bend'): 0.5,('CVG', 'Des Moines'): 0.5,	('AFW', 'Des Moines'): 0.5,('CVG', 'Wichita'): 0.5,	('AFW', 'Wichita'): 0.5,	
    ('CVG', 'New Orleans'): 0.5,('AFW', 'New Orleans'): 0.5,('CVG', 'Baltimore'): 0.5,	('AFW', 'Baltimore'): 0.5,('CVG', 'Minneapolis'): 0.5,('AFW', 'Minneapolis'): 0.5,('CVG', 'Kansas City'): 0.5,('AFW', 'Kansas City'): 0.5,	
    ('CVG', 'St. Louis'): 0.5,('AFW', 'St. Louis'): 0.5,('CVG', 'Omaha'): 0.5,('AFW', 'Omaha'): 0.5,('CVG', 'Manchester'): 0.5,('AFW', 'Manchester'): 0.5,('CVG', 'Albuquerque'): 0.5,('AFW', 'Albuquerque'): 0.5,	
    ('CVG', 'New York'): 0.5,('AFW', 'New York'): 0.5, ('CVG', 'Charlotte'): 0.5,('AFW', 'Charlotte'): 0.5,('CVG', 'Toledo'): 0.5,('AFW', 'Toledo'): 0.5,('CVG', 'Wilmington'): 0.5,	('AFW', 'Wilmington'): 0.5,	
    ('CVG', 'Portland'): 0.5,('AFW', 'Portland'): 0.5,('CVG', 'Allentown'): 0.5,('AFW', 'Allentown'): 0.5,('CVG', 'Pittsburgh'): 0.5,('AFW', 'Pittsburgh'): 0.5,('CVG', 'San Juan'): 0.5,('AFW', 'San Juan'): 0.5,	
    ('CVG', 'Nashville'): 0.5,('AFW', 'Nashville'): 0.5,('CVG', 'Austin'): 0.5,	('AFW', 'Austin'): 0.25,('CVG', 'Dallas/Fort Worth'): 0.5,('CVG', 'Houston'): 0.5,('AFW', 'Houston'): 0.25,('CVG', 'San Antonio'): 0.5,
    ('AFW', 'San Antonio'): 0.25,('CVG', 'Richmond'): 0.5,('AFW', 'Richmond'): 0.5,('CVG', 'Seattle/Tacoma'): 0.5,	('AFW', 'Seattle/Tacoma'): 0.5,	('CVG', 'Spokane'): 0.5,	('AFW', 'Spokane'): 0.5
}

# Focus City to center costs (Our z variables)

costs_z = {
    ('Leipzig', 'Paris'): 0.5,('Leipzig', 'Cologne'): 0.5,('Leipzig', 'Hanover'): 0.5,('Leipzig', 'Bengaluru'): 1.5,('Leipzig', 'Coimbatore'): 1.5,('Leipzig', 'Delhi'): 1.5,('Leipzig', 'Mumbai'): 1.5,	
    ('Leipzig', 'Cagliari'): 0.5,('Leipzig', 'Catania'): 0.5,('Leipzig', 'Milan'): 0.5,('Leipzig', 'Rome'): 0.5,('Leipzig', 'Katowice'): 0.5,('Leipzig', 'Barcelona'): 0.5,('Leipzig', 'Madrid'): 0.5,	
    ('Leipzig', 'Castle Donington'): 0.5,('Leipzig', 'London'): 0.75,('San Bernardino', 'Mobile'): 0.5,('San Bernardino', 'Anchorage'): 0.7,('San Bernardino', 'Fairbanks'): 0.7,('San Bernardino', 'Phoenix'): 0.5,
	('San Bernardino', 'Sacramento'): 0.5,('San Bernardino', 'San Francisco'): 0.5,('San Bernardino', 'Stockton'): 0.5,('San Bernardino', 'Denver'): 0.5,('Leipzig', 'Hartford'): 1.5,('San Bernardino', 'Hartford'): 0.5,
	('San Bernardino', 'Miami'): 0.7,('San Bernardino', 'Lakeland'): 0.7,('San Bernardino', 'Tampa'): 0.7,('San Bernardino', 'Atlanta'): 0.6,('San Bernardino', 'Honolulu'): 0.5,('San Bernardino', 'Kahului/Maui'): 0.5,
	('San Bernardino', 'Kona'): 0.5,('San Bernardino', 'Chicago'): 0.5,('San Bernardino', 'Rockford'): 0.5,('San Bernardino', 'Fort Wayne'): 0.5,('San Bernardino', 'South Bend'): 0.5,('San Bernardino', 'Des Moines'): 0.5,
	('San Bernardino', 'Wichita'): 0.5,('San Bernardino', 'New Orleans'): 0.5,('Leipzig', 'Baltimore'): 1.5,('San Bernardino', 'Baltimore'): 0.7,('San Bernardino', 'Minneapolis'): 0.5,('San Bernardino', 'Kansas City'): 0.5,
	('San Bernardino', 'St. Louis'): 0.5,('San Bernardino', 'Omaha'): 0.5,('Leipzig', 'Manchester'): 1.5,('San Bernardino', 'Manchester'): 0.7,('San Bernardino', 'Albuquerque'): 0.5,('Leipzig', 'New York'): 1.6,	
    ('San Bernardino', 'New York'): 0.7,('San Bernardino', 'Charlotte'): 0.7,('San Bernardino', 'Toledo'): 0.5,('San Bernardino', 'Wilmington'): 0.7,('San Bernardino', 'Portland'): 0.5,('Leipzig', 'Allentown'): 1.5,	
    ('San Bernardino', 'Allentown'): 0.7,('San Bernardino', 'Pittsburgh'): 0.6,('San Bernardino', 'San Juan'): 1,('San Bernardino', 'Nashville'): 0.5,('San Bernardino', 'Austin'): 0.5,('San Bernardino', 'Dallas/Fort Worth'): 0.5,
	('San Bernardino', 'Houston'): 0.5,('San Bernardino', 'San Antonio'): 0.5,('San Bernardino', 'Richmond'): 0.7,('San Bernardino', 'Seattle/Tacoma'): 0.5,('San Bernardino', 'Spokane'): 0.5, ('Leipzig', 'Hyderabad'): 1.6
}





66


In [3]:
# Step 1: Create the optimization problem:
prob = LpProblem("Amazon_Distribution_Cost_Minimization", LpMinimize)

In [4]:
# Step 2: Create decision variables.  X represents shipment from hub to focus, Y represents shipment from hub to centers, and Z represents shipment from focus to centers.
# Since a fraction of a ton has practical meaning, we are presuming all variables to be continuous and not discrete integers ... thus our use of PuLP
x_vars = LpVariable.dicts("x", costs_x.keys(), lowBound=0)

y_vars = LpVariable.dicts("y", costs_y.keys(), lowBound=0)

z_vars = LpVariable.dicts("z", costs_z.keys(), lowBound=0)

print(f"Number of x variables (hub to focus): {len(x_vars)}")
print(f"Number of y variables (hub to centers): {len(y_vars)}")
print(f"Number of z variables (focus to centers): {len(z_vars)}")

print("\nx variables (hub to focus city):")
for route in x_vars:
    print(f"   {route}")

print("\ny variables (hub to center) - showing first 10:")
for route in list(y_vars.keys())[:10]:
    print(f"   {route}")
print(f" ... and {len(y_vars) - 10} more")

print("\nz variables (focus to center) - showing first 10:")
for route in list(z_vars.keys())[:10]:
    print(f"   {route}")
print(f" ... and {len(z_vars) - 10} more")

# Check what's in your costs_y
print(f"Routes defined in costs_y dictionary: {len(costs_y)}")
print(f"Variables created in y_vars: {len(y_vars)}")

Number of x variables (hub to focus): 3
Number of y variables (hub to centers): 106
Number of z variables (focus to centers): 68

x variables (hub to focus city):
   ('CVG', 'Leipzig')
   ('CVG', 'San Bernardino')
   ('AFW', 'San Bernardino')

y variables (hub to center) - showing first 10:
   ('CVG', 'Paris')
   ('CVG', 'Cologne')
   ('CVG', 'Hanover')
   ('CVG', 'Cagliari')
   ('CVG', 'Catania')
   ('CVG', 'Milan')
   ('CVG', 'Rome')
   ('CVG', 'Katowice')
   ('CVG', 'Barcelona')
   ('CVG', 'Madrid')
 ... and 96 more

z variables (focus to center) - showing first 10:
   ('Leipzig', 'Paris')
   ('Leipzig', 'Cologne')
   ('Leipzig', 'Hanover')
   ('Leipzig', 'Bengaluru')
   ('Leipzig', 'Coimbatore')
   ('Leipzig', 'Delhi')
   ('Leipzig', 'Mumbai')
   ('Leipzig', 'Cagliari')
   ('Leipzig', 'Catania')
   ('Leipzig', 'Milan')
 ... and 58 more
Routes defined in costs_y dictionary: 106
Variables created in y_vars: 106


In [5]:
# Step three creating the objective function

prob += (
    lpSum([costs_x[route] * x_vars[route] for route in x_vars]) + 
    lpSum([costs_y[route] * y_vars[route] for route in y_vars]) +
    lpSum([costs_z[route] * z_vars[route] for route in z_vars]), 
    "Total_Shipping_Cost"
)

In [6]:
# Step 4 Adding constraints

# First Demand constraints. Each center must receive exactly its demand
for center in centers:
    prob +=(
        lpSum([y_vars[(hub, center)] for hub in hubs if (hub, center) in y_vars]) +                 # add in from hubs
        lpSum([z_vars[(focus, center)] for focus in focus_cities if (focus,center) in z_vars])      # add in from focus 
        == demand[center],
        f"Demand_{center}"
    )

print(f"Total constraints: {len(prob.constraints)}\n")
for i, (name, constraint) in enumerate(list(prob.constraints.items())):
    print(f"{name}: {constraint}")

Total constraints: 66

Demand_Hyderabad: z_('Leipzig',_'Hyderabad') = -0.0
Demand_Paris: y_('CVG',_'Paris') + z_('Leipzig',_'Paris') = 6500.0
Demand_Cologne: y_('CVG',_'Cologne') + z_('Leipzig',_'Cologne') = 640.0
Demand_Hanover: y_('CVG',_'Hanover') + z_('Leipzig',_'Hanover') = 180.0
Demand_Bengaluru: z_('Leipzig',_'Bengaluru') = 9100.0
Demand_Coimbatore: z_('Leipzig',_'Coimbatore') = 570.0
Demand_Delhi: z_('Leipzig',_'Delhi') = 19000.0
Demand_Mumbai: z_('Leipzig',_'Mumbai') = 14800.0
Demand_Cagliari: y_('CVG',_'Cagliari') + z_('Leipzig',_'Cagliari') = 90.0
Demand_Catania: y_('CVG',_'Catania') + z_('Leipzig',_'Catania') = 185.0
Demand_Milan: y_('CVG',_'Milan') + z_('Leipzig',_'Milan') = 800.0
Demand_Rome: y_('CVG',_'Rome') + z_('Leipzig',_'Rome') = 1700.0
Demand_Katowice: y_('CVG',_'Katowice') + z_('Leipzig',_'Katowice') = 170.0
Demand_Barcelona: y_('CVG',_'Barcelona') + z_('Leipzig',_'Barcelona') = 2800.0
Demand_Madrid: y_('CVG',_'Madrid') + z_('Leipzig',_'Madrid') = 3700.0
Demand_Ca

In [7]:
# Second set of constraints..... Capacity constraints

# Hub Capacity: Total outflow from each hub <= capacity
for hub in hubs:
    prob += (
        lpSum([x_vars[(hub,focus)] for focus in focus_cities if (hub, focus) in x_vars]) +
        lpSum([y_vars[(hub, center)] for center in centers if (hub, center) in y_vars])
        <= hub_capacity[hub],
        f"Hub_Capacity_{hub}"
    )

# Focus city capacity: total throughput <= capacity
for focus in focus_cities:
    prob += (
        lpSum([x_vars[(hub, focus)] for hub in hubs if (hub, focus) in x_vars]) 
        <= focus_capacity[focus],
        f"Focus_Capacity_{focus}"
    )

# Show hub capacity constraints
hub_constraints = {k: v for k, v in prob.constraints.items() if 'Hub_Capacity' in k}
print(f"Hub capacity constraints ({len(hub_constraints)}):")
for name, constraint in hub_constraints.items():
    print(f"  {name}: {constraint}")

print("\n")

# Show focus capacity constraints
focus_constraints = {k: v for k, v in prob.constraints.items() if 'Focus_Capacity' in k}
print(f"Focus capacity constraints ({len(focus_constraints)}):")
for name, constraint in focus_constraints.items():
    print(f"  {name}: {constraint}")

Hub capacity constraints (2):
  Hub_Capacity_CVG: x_('CVG',_'Leipzig') + x_('CVG',_'San_Bernardino') + y_('CVG',_'Albuquerque') + y_('CVG',_'Allentown') + y_('CVG',_'Anchorage') + y_('CVG',_'Atlanta') + y_('CVG',_'Austin') + y_('CVG',_'Baltimore') + y_('CVG',_'Barcelona') + y_('CVG',_'Cagliari') + y_('CVG',_'Castle_Donington') + y_('CVG',_'Catania') + y_('CVG',_'Charlotte') + y_('CVG',_'Chicago') + y_('CVG',_'Cologne') + y_('CVG',_'Dallas_Fort_Worth') + y_('CVG',_'Denver') + y_('CVG',_'Des_Moines') + y_('CVG',_'Fairbanks') + y_('CVG',_'Fort_Wayne') + y_('CVG',_'Hanover') + y_('CVG',_'Hartford') + y_('CVG',_'Houston') + y_('CVG',_'Kansas_City') + y_('CVG',_'Katowice') + y_('CVG',_'Lakeland') + y_('CVG',_'London') + y_('CVG',_'Los_Angeles') + y_('CVG',_'Madrid') + y_('CVG',_'Manchester') + y_('CVG',_'Miami') + y_('CVG',_'Milan') + y_('CVG',_'Minneapolis') + y_('CVG',_'Mobile') + y_('CVG',_'Nashville') + y_('CVG',_'New_Orleans') + y_('CVG',_'New_York') + y_('CVG',_'Omaha') + y_('CVG',_'On

In [8]:
# Finally adding Flow Conservation constraints at focus cities.  flow into a focus city must equal flow out of a focus city.... 
for focus in focus_cities:
    prob += (
        lpSum([x_vars[(hub, focus)] for hub in hubs if (hub, focus) in x_vars]) # inflow from hubs
        ==
        lpSum([z_vars[(focus, center)] for center in centers if (focus, center) in z_vars]),  # out flow from focus
        f"Flow_Conservation_{focus}"
    )
    
flow_constraints = {k: v for k, v in prob.constraints.items() if 'Flow_Conservation' in k}
print(f"Flow conservation constraints ({len(flow_constraints)}):")
for name, constraint in flow_constraints.items():
    print(f"  {name}: {constraint}")


Flow conservation constraints (2):
  Flow_Conservation_Leipzig: x_('CVG',_'Leipzig') - z_('Leipzig',_'Allentown') - z_('Leipzig',_'Baltimore') - z_('Leipzig',_'Barcelona') - z_('Leipzig',_'Bengaluru') - z_('Leipzig',_'Cagliari') - z_('Leipzig',_'Castle_Donington') - z_('Leipzig',_'Catania') - z_('Leipzig',_'Coimbatore') - z_('Leipzig',_'Cologne') - z_('Leipzig',_'Delhi') - z_('Leipzig',_'Hanover') - z_('Leipzig',_'Hartford') - z_('Leipzig',_'Hyderabad') - z_('Leipzig',_'Katowice') - z_('Leipzig',_'London') - z_('Leipzig',_'Madrid') - z_('Leipzig',_'Manchester') - z_('Leipzig',_'Milan') - z_('Leipzig',_'Mumbai') - z_('Leipzig',_'New_York') - z_('Leipzig',_'Paris') - z_('Leipzig',_'Rome') = -0.0
  Flow_Conservation_San_Bernardino: x_('AFW',_'San_Bernardino') + x_('CVG',_'San_Bernardino') - z_('San_Bernardino',_'Albuquerque') - z_('San_Bernardino',_'Allentown') - z_('San_Bernardino',_'Anchorage') - z_('San_Bernardino',_'Atlanta') - z_('San_Bernardino',_'Austin') - z_('San_Bernardino',_'Ba

In [9]:
# Final step, solve the problem and check the status
prob.solve()

print("Status:", LpStatus[prob.status])

Status: Optimal


In [11]:
# Display solution status and objective value
print("Status:", LpStatus[prob.status])
print(f"Total Cost: ${value(prob.objective):,.2f}\n")

# Show which routes are actually used (non-zero flows)
print("=== ROUTES USED ===\n")

print("Hub to Focus City:")
for route, var in x_vars.items():
    if var.varValue > 0:
        print(f"  {route[0]} → {route[1]}: {var.varValue:,.2f} tons")

print("\nHub to Center (showing routes with flow > 0):")
active_y = [(route, var.varValue) for route, var in y_vars.items() if var.varValue > 0]
for route, flow in sorted(active_y, key=lambda x: -x[1])[:10]:  # Top 10
    print(f"  {route[0]} → {route[1]}: {flow:,.2f} tons")
print(f"  ... and {len(active_y) - 10} more routes")

print("\nFocus to Center (showing routes with flow > 0):")
active_z = [(route, var.varValue) for route, var in z_vars.items() if var.varValue > 0]
for route, flow in sorted(active_z, key=lambda x: -x[1])[:10]:  # Top 10
    print(f"  {route[0]} → {route[1]}: {flow:,.2f} tons")
print(f"  ... and {len(active_z) - 10} more routes")

Status: Optimal
Total Cost: $199,476.25

=== ROUTES USED ===

Hub to Focus City:
  CVG → Leipzig: 43,470.00 tons

Hub to Center (showing routes with flow > 0):
  CVG → New York: 8,035.00 tons
  CVG → Los Angeles: 7,200.00 tons
  CVG → London: 6,700.00 tons
  CVG → Paris: 6,500.00 tons
  AFW → Chicago: 5,100.00 tons
  CVG → Madrid: 3,700.00 tons
  AFW → Miami: 3,400.00 tons
  CVG → Dallas/Fort Worth: 3,300.00 tons
  AFW → Houston: 3,300.00 tons
  AFW → New York: 3,165.00 tons
  ... and 52 more routes

Focus to Center (showing routes with flow > 0):
  Leipzig → Delhi: 19,000.00 tons
  Leipzig → Mumbai: 14,800.00 tons
  Leipzig → Bengaluru: 9,100.00 tons
  Leipzig → Coimbatore: 570.00 tons
  ... and -6 more routes


In [13]:
# Verify hub capacity constraints
print("=== HUB CAPACITY CHECK ===")
for hub in hubs:
    outflow = sum([x_vars[(hub, focus)].varValue for focus in focus_cities if (hub, focus) in x_vars])
    outflow += sum([y_vars[(hub, center)].varValue for center in centers if (hub, center) in y_vars])
    print(f"{hub}: {outflow:,.2f} / {hub_capacity[hub]:,} tons ({outflow/hub_capacity[hub]*100:.1f}% utilized)")

# Verify focus city flow conservation
print("\n=== FOCUS CITY FLOW CONSERVATION ===")
for focus in focus_cities:
    inflow = sum([x_vars[(hub, focus)].varValue for hub in hubs if (hub, focus) in x_vars])
    outflow = sum([z_vars[(focus, center)].varValue for center in centers if (focus, center) in z_vars])
    print(f"{focus}: IN = {inflow:,.2f}, OUT = {outflow:,.2f}, Balanced = {abs(inflow - outflow) < 0.01}")

# Verify demand satisfaction for each center
print("\n=== DEMAND SATISFACTION CHECK ===")
demand_errors = []
for center in centers:
    received_from_hubs = sum([y_vars[(hub, center)].varValue for hub in hubs if (hub, center) in y_vars])
    received_from_focus = sum([z_vars[(focus, center)].varValue for focus in focus_cities if (focus, center) in z_vars])
    total_received = received_from_hubs + received_from_focus
    
    if abs(total_received - demand[center]) > 0.01:  # Allow tiny floating point errors
        demand_errors.append((center, demand[center], total_received))

if demand_errors:
    print("ERRORS FOUND:")
    for center, needed, received in demand_errors:
        print(f"  {center}: needed {needed:,.2f}, received {received:,.2f}")
else:
    print("All centers received exactly their demanded amounts")
    
# Show a few examples
print("\nSample verification:")
sample_centers = ['New York', 'Delhi', 'Paris', 'Hyderabad']
for center in sample_centers:
    if center in centers:
        received_from_hubs = sum([y_vars[(hub, center)].varValue for hub in hubs if (hub, center) in y_vars])
        received_from_focus = sum([z_vars[(focus, center)].varValue for focus in focus_cities if (focus, center) in z_vars])
        total = received_from_hubs + received_from_focus
        print(f"  {center}: demanded {demand[center]:,.2f}, received {total:,.2f}")

=== HUB CAPACITY CHECK ===
CVG: 95,650.00 / 95,650 tons (100.0% utilized)
AFW: 38,097.00 / 44,350 tons (85.9% utilized)

=== FOCUS CITY FLOW CONSERVATION ===
Leipzig: IN = 43,470.00, OUT = 43,470.00, Balanced = True
San Bernardino: IN = 0.00, OUT = 0.00, Balanced = True

=== DEMAND SATISFACTION CHECK ===
All centers received exactly their demanded amounts

Sample verification:
  New York: demanded 11,200.00, received 11,200.00
  Delhi: demanded 19,000.00, received 19,000.00
  Paris: demanded 6,500.00, received 6,500.00
  Hyderabad: demanded 0.00, received 0.00
